In [2]:
from bs4 import BeautifulSoup
import requests
import re
import os
import pandas as pd

In [ ]:
os.chdir('your directory')

In [ ]:
raw_data = pd.read_csv('full_table/aggregated.csv', dtype=str, usecols=['date','title','link','content'])
index = pd.DataFrame({'index' : [i for i in range(2500)]}) 
raw_data.insert(0, 'index', index)
content_df = raw_data['content']

In [ ]:
from nltk import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords

# NLTK의 불용어(Stopword) 
stops = set(stopwords.words("english"))

Loughran and McDonald(2011) Financial Dictionary 사용: 긍정 및 부정 어휘 

In [ ]:
dic_pos = pd.read_csv("test/fin_pos.txt", names=['word_list'], quoting=3)
dic_pos = dic_pos['word_list'].tolist() 
dic_neg = pd.read_csv("test/fin_neg.txt", names=['word_list'], quoting=3)
dic_neg = dic_neg['word_list'].tolist() 

In [ ]:
def calculate_sentiment(text):   
    
    # 클렌징 작업 
    text = text[:-250]
    text = re.sub(r'(?=googletag)(.*)(?=;).+', '', text)
    text = re.sub('[^a-zA-Z\s]', '', text)
    text = text.lower() 
    text = ' '.join(text.split())
    text = text[0:3499]      # 리스트의 모든 문자열을 평균 길이(3500)로 통일 
    

    # 토크나이징 및 불용어 제거   
    tokenized_text = sent_tokenize(text)  #nltk sent tokenizing
    word_list = [] 
    
    for each in tokenized_text: 
        word_list.extend(word_tokenize(text)) 

    word_list = [w for w in word_list if not w in stops]
    

    # 감성지수(Sentiment Index) 계산
  
    # Positive
    pos_word_list = [word for word in word_list if word in dic_pos]     

    # Negative
    neg_word_list = [word for word in word_list if word in dic_neg]     
  
    # Count 
    total_sent = len(pos_word_list) - len(neg_word_list)
    total_length = len(pos_word_list) + len(neg_word_list)
    
    if total_length == 0:       # Financial Dictionary와 하나도 매칭이 안되는 경우
        
        return 0.00 
    
    else: 
        sent_index = (total_sent / total_length)
    
        return '{:.2f}'.format(sent_index)

In [ ]:
sent_list = []
for i in range(0,2500):
    text = content_df[i] 
    sent_list.append(calculate_sentiment(text)) 

sent_df = pd.DataFrame({'index' : [i for i in range(2500)], 'sentiment' : sent_list}, dtype=str)

모든 변수 합치기. Outer merge by index

In [ ]:
left_data = raw_data
right_data = sent_df     
final_df = pd.merge(left_data, right_data, how='outer', on='index')  
final_df.to_csv('full_table/sent.csv',sep=',', encoding='utf-8', index=False)